In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
from peft import PeftModel
import gradio as gr
from accelerate import infer_auto_device_map, init_empty_weights

# --- 1. Definujte cesty a konfigurace ---
base_model_name_or_path = "/home/luk-hrd/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.1/snapshots/2dcff66eac0c01dc50e4c41eea959968232187fe"
lora_adapter_path = "mistral-7b-quantum-instruct-v1/final_adapter" 

# --- 2. Upravená konfigurace kvantizace ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True
)

# --- 3. Načtení modelu s vlastní device_map ---
print("Načítám kvantizovaný model a tokenizer...")

# Vytvoříme si vlastní device_map místo "auto"
with init_empty_weights():
    model_empty = AutoModelForCausalLM.from_pretrained(
        base_model_name_or_path, 
        config=AutoConfig.from_pretrained(base_model_name_or_path)
    )

# !!! ZDE JE FINÁLNÍ OPRAVA: Odstraněn argument 'quantization_config' !!!
device_map = infer_auto_device_map(
    model_empty,
    dtype=torch.bfloat16,
    no_split_module_classes=["MistralDecoderLayer"]
)
print("Vygenerovaná device_map:")
print(device_map)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    quantization_config=bnb_config, # Tento argument zde zůstává, je to správně
    device_map=device_map,
)

# Zbytek kódu je stejný
tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Aplikuji LoRA adaptér...")
model = PeftModel.from_pretrained(model, lora_adapter_path)
model.eval()

print("Model je připraven!")


# --- Funkce pro Gradio a spuštění UI (beze změny) ---
def chat_with_model_gradio(message, history):
    history_prompt = ""
    for user_msg, assistant_msg in history:
        history_prompt += f"[INST] {user_msg} [/INST] {assistant_msg}</s>"

    final_prompt = f"<s>{history_prompt}[INST] {message} [/INST]"

    inputs = tokenizer(final_prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    clean_response = response.split("[/INST]")[-1].strip()
    
    return clean_response

iface = gr.ChatInterface(
    fn=chat_with_model_gradio,
    title="PhD AI Assistant (Quantum Mechanics)",
    description="Chat with me about quantum mechanics and computing.",
    theme="soft",
     examples=[
        ["Explain the Bell state in detail."],
        ["Write a Qiskit code snippet to create a GHZ state."],
    ]
)

iface.launch()

/home/luk-hrd/miniconda3/envs/mistral-lora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Načítám kvantizovaný model a tokenizer...


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.26s/it]


Vygenerovaná device_map:
OrderedDict([('model.embed_tokens', 0), ('model.layers.0', 0), ('model.layers.1', 0), ('model.layers.2', 0), ('model.layers.3', 0), ('model.layers.4', 0), ('model.layers.5', 0), ('model.layers.6', 0), ('model.layers.7', 0), ('model.layers.8', 0), ('model.layers.9', 0), ('model.layers.10', 0), ('model.layers.11', 0), ('model.layers.12', 0), ('model.layers.13', 0), ('model.layers.14', 0), ('model.layers.15', 0), ('model.layers.16', 0), ('model.layers.17', 'cpu'), ('model.layers.18', 'cpu'), ('model.layers.19', 'cpu'), ('model.layers.20', 'cpu'), ('model.layers.21', 'cpu'), ('model.layers.22', 'cpu'), ('model.layers.23', 'cpu'), ('model.layers.24', 'cpu'), ('model.layers.25', 'cpu'), ('model.layers.26', 'cpu'), ('model.layers.27', 'cpu'), ('model.layers.28', 'cpu'), ('model.layers.29', 'cpu'), ('model.layers.30', 'cpu'), ('model.layers.31', 'cpu'), ('model.norm', 'cpu'), ('model.rotary_emb', 'cpu'), ('lm_head', 'cpu')])


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.43s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


Aplikuji LoRA adaptér...


Some parameters are on the meta device because they were offloaded to the cpu.
/home/luk-hrd/miniconda3/envs/mistral-lora/lib/python3.10/site-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Model je připraven!
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


/home/luk-hrd/miniconda3/envs/mistral-lora/lib/python3.10/site-packages/transformers/generation/utils.py:2479: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [2]:
import torch

if torch.cuda.is_available():
    print("GPU je dostupná!")
    print(f"Název zařízení: {torch.cuda.get_device_name(0)}")
    print(f"Celková paměť: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Volná paměť: {torch.cuda.mem_get_info(0)[0] / 1e9:.2f} GB")
else:
    print("POZOR: GPU není dostupná. PyTorch používá pouze CPU.")

GPU je dostupná!
Název zařízení: NVIDIA GeForce RTX 4070 Laptop GPU
Celková paměť: 8.32 GB
Volná paměť: 4.52 GB
